# Математическое моделирование распределения легкой примеси в приземном слое атмосферы, описанное в рамках теории подобия 

In [1]:
import math
from numba import njit
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

### Определяем функции теории подобия МО

In [2]:
def psi(z, L, z0):
    return 1./(1.+4.7*(z+z0)) if L>0 else (1+ (1 if L>0 else -1)*15*z) ** 0.25


def iks(z, L):
    return (1+ (1 if L>0 else -1)*15*z) ** 0.25


def f_MO(z, L):
    return (np.log(z*L) + 4.7*z if L>0 else 
            (np.log(z*abs(L)) - 2.*np.log(0.5*(1+iks(z,L))) -
            np.log(0.5*(1+iks(z,L))) + 2.* np.arctan(iks(z,L))))


def u_MO(z, L, z0):
    return 1./KAPPA*(f_MO(z,L)-f_MO(z0,L))


def k_MO(z, L, z0):
    return KAPPA*(z+z0)*psi(z,L, z0)

### Решение конечно-разностной системы уравнений методом  прогонки

In [3]:
@njit
def progon(A, C, B, F, Y):

    alfa = np.zeros(n + 1)
    beta = np.zeros(n + 1)

    # прямой ход метода прогонки
    # вычисление коэффициентов прогонки
    alfa[0] = B[0] / C[0]
    beta[0] = F[0] / C[0]
    for i in range(1, n + 1):
        alfa[i] = B[i] / (C[i] - alfa[i-1]*A[i])
        beta[i] = (A[i]*beta[i-1] + F[i]) / (C[i] - alfa[i-1]*A[i])

    # обратный ход метода прогонки
    Y[i] = beta[n]
    for i in range(n - 1, -1, -1):
        Y[i] = alfa[i]*Y[i+1] + beta[i]

### Вычисление по явной схеме

In [4]:
@njit
def explicit_scheme():
    # вычисление по явной схеме
    for j in range(m):
        for i in range(1, n-1):
            sigma = dx / (uf[i] * dz**2)
            k_minus = 0.5*(kf[i] + kf[i-1])
            k_plus = 0.5*(kf[i] + kf[i+1])
            #c[i, j+1] = (sigma * k_plus * c[i+1, j] + (1- sigma * (k_minus + k_plus)) * c[i, j]
             #           + sigma * k_minus * c[i-1, j])
            c[i, j+1] = (sigma * (k_plus*c[i+1, j] + k_minus*c[i-1, j]) +
                           (1 - sigma*(k_minus+k_plus)) * c[i, j])

        c[0, j+1] = c[1, j+1]
        c[n, j] = 0

    for j in range(1, m):
        for i in range(n):
            cpoint[i,j] = c[i,j] / (2 * math.sqrt(math.pi*k0*x[j]))
    # вычисление по явной схеме выполнено

### Вычисление по неявной схеме

In [5]:
@njit
def implicit_scheme(cn):
    #global cn
    # определение массивов для коэффициентов системы 
    # для метода прогонки в чисто неявной схеме
    A = np.zeros(n+1)
    C = np.zeros(n+1)
    B = np.zeros(n+1)
    F = np.zeros(n+1)
    # вычисление по чисто неявной схеме
    for j in range(1, m+1):
        # вычисление коэффциентов системы уравнений
        C[0], B[0], F[0] = 1., 1., 0.
        C[n], A[n], F[n] = 1., 0., 0.
        for i in range(1,n):
            sigma = uf[i] * dz**2 / dx
            k_minus = 0.5 * (kf[i] + kf[i-1])
            k_plus = 0.5 * (kf[i] + kf[i+1])
            A[i] = k_minus
            C[i] = k_plus + k_minus + sigma
            B[i] = k_plus
            F[i] = sigma * cn[i, j-1] #*0.983
        progon(A, C, B, F, cn[:, j])
    # вычисление по неявной схеме выполнено

### Вычисление по схеме с весами $\sigma=0,5$

In [6]:
@njit
def Krank_Nikolson():
    # вычисление по схеме Кранка-Николсона
    for j in range(1, m+1):
        # вычисление коэффциентов системы уравнений
        C[0] = 1.; B[0] = 1.; F[0] = 0.
        C[n] = 1.; A[n] = 0.; F[n] = 0.
        for i in range(1,n):
            sigma = uf[i] * dz**2 / dx
            k_minus = 0.5 * (kf[i] + kf[i-1])
            k_plus = 0.5 * (kf[i] + kf[i+1])
            A[i] = k_minus
            C[i] = k_plus + k_minus + 2*sigma
            B[i] = k_plus
            F[i] = (2*sigma*ck[i,j-1] + k_plus*(ck[i+1,j-1]-ck[i,j-1])
            - k_minus*(ck[i,j-1]-ck[i-1,j-1]))


        progon(A, C, B, F, ck[:, j])

# вычисление по схеме Кранка-Николсона выполнено

### Задаем входные параметры модели

In [7]:
# Постоянная Кармана
KAPPA = 0.4187
# Масштаб дины Монина-Обухова
L = 100.
# Источник мощностью M на высоте H
M = 10.
# Безразмерная высота соответствует источнику на высоте 10 м
H = 10. / abs(L) 
# Скорость ветра и коэффициент турбулентности
U = 10.
K = 5.
# Скорость ветра на высоте источника - пока равна U
uH = U
# Динамическая скорость
UDIN = 0.5
# Коэффициент для перехода к безразмерной концентрации
C = M / (UDIN*abs(L))

### Определяем границы расчетной области, шаг и сетку

In [8]:
# Шаг сетки в безразмерных переменных
dz = 0.05
dx = 0.01
# Правая граница расчетной области в метрах
right = 3000.

# Безразмерная правая граница
xmax = right / L   # + 0.00001

# Безразмерный параметр шероховатости
z0 = 0.0002
# Верхняя граница расчетной области
zmax = 1. + z0
# Мссив значений x и z
#x = np.arange(0.001, xmax+dx, dx)
x = np.arange(0., xmax+dx, dx)
z = np.arange(z0, zmax+dz, dz)
# Количество шагов
n = z.size - 1
m = x.size - 1

### Определяем массивы значений коэффициентов турбулентности

In [9]:
# В общем случае скорость ветра и к-т турбулентности - функции от z
#uf = np.ones(n+1)
#kf = np.ones(n+1)
# Но сейчас задаем их константами
#uf = uf * u
#kf = kf * K

In [10]:
kf = np.zeros(n)
uf = np.zeros(n)

In [11]:
kf = k_MO(z, L, z0)
kf

array([0.00016717, 0.01706106, 0.0285604 , 0.03689333, 0.04320941,
       0.04816181, 0.05214915, 0.05542846, 0.05817296, 0.06050361,
       0.06250745, 0.06424873, 0.06577587, 0.06712609, 0.06832844,
       0.06940597, 0.07037714, 0.07125696, 0.07205775, 0.07278969,
       0.07346129])

In [12]:
uf = u_MO(z, L, z0)
uf

array([ 0.        , 13.75794827, 15.96992143, 17.49798341, 18.74553327,
       19.83926083, 20.83565042, 21.76484913, 22.64485869, 23.48729363,
       24.30008582, 25.08889363, 25.8578955 , 26.61026492, 27.34846888,
       28.07446322, 28.78982472, 29.49584315, 30.19358699, 30.88395128,
       31.56769327])

In [13]:
# массив cn - решение задачи для линейного источника в неявной задаче
cn = np.zeros((n+1, m+1))

### Задание начального условия

In [14]:
# Определяем номер узла, соответствующего источнику
i = 1
while i <= n :
    if z[i-1] <= H and H <= z[i]:
        break
    i += 1

iH = i
# И задаем начальное условие в этом узле
#cn[iH, 0] = M/(uH * dz)
cn[iH, 0] = 1./(uf[iH]*dz)

In [15]:
implicit_scheme(cn)

### Функция графического вывода результатов

In [16]:
def mv_graphics_stat(graphs_visible, my_title,
                     mv_title_x, mv_title_y,
                     file_name,
                     xaxis_min, xaxis_max,
                     mv_legend_x, mv_legend_anchor):
    fig = go.Figure(data = graphs_visible,
                    layout_xaxis_range = [xaxis_min,
                                          xaxis_max])
    fig.layout.font.family = 'Times'
    fig.layout.font.size = 14
    
    fig.update_layout(xaxis_title = mv_title_x,
                  yaxis_title = mv_title_y,
                  legend=dict(x = mv_legend_x,
                              xanchor =
                              mv_legend_anchor),
                  autosize=False,
                  width=600, height=400)
    fig.update_layout(margin = 
                      dict(l=25, r=0, t=0, b=25))

    
    fig.show()

In [17]:
graphs_visible = [go.Scatter(visible = True, x=x*L, y=cn[1,:]*C, 
                               name='{:,.1f} м'.format(z[1]*L).replace(",", " "),
                               line_dash = 'dot', line_color = '#ffa306'),
                    go.Scatter(visible = True, x=x*L, y=cn[4,:]*C, 
                               name='{:,.1f} м'.format(z[4]*L).replace(",", " "),
                               line_dash = 'dot', line_color = '#ffa306'),
                    go.Scatter(visible = True, x=x*L, y=cn[8,:]*C, 
                               name='{:,.0f} м'.format(z[8]*L).replace(",", " "),
                                line_dash = 'solid', line_color = '#ffa306'),
                    go.Scatter(visible = True,x=x*L, y=cn[12,:]*C,
                               name='{:,.0f} м'.format(z[12]*L).replace(",", " "),
                               line_dash = 'dot', line_color = '#2a11ff'),
                 ]


my_title = "Приземные концентрации на разных высотах"
mv_title_x = "$$t, ч$$"
mv_title_y = "$$c,  ^\circ С$$"
file_name = "diary_tau"
xaxis_min = 0.
xaxis_max = 3000.
mv_legend_x = 1.
mv_legend_anchor = "right"

mv_graphics_stat(graphs_visible, my_title,
                 mv_title_x, mv_title_y,
                 file_name,
                 xaxis_min, xaxis_max,
                 mv_legend_x, mv_legend_anchor)

In [18]:
graphs_visible = [go.Scatter(visible=True, x=uf*UDIN, y=z, 
                               name='{:,.1f} м'.format(z[4]).replace(",", " "),
                               line_dash = 'dot', line_color = '#ffa306')
                 ]


my_title = "Приземные концентрации на разных высотах"
mv_title_x = "$$u, м/с$$"
mv_title_y = "$$z,  м$$"
file_name = "diary_tau"
xaxis_min = 0.
xaxis_max = 33.
mv_legend_x = 1.
mv_legend_anchor = "right"

mv_graphics_stat(graphs_visible, my_title,
                 mv_title_x, mv_title_y,
                 file_name,
                 xaxis_min, xaxis_max,
                 mv_legend_x, mv_legend_anchor)

In [19]:
graphs_visible = [go.Scatter(visible=True, x=kf, y=z, 
                               name='{:,.1f} м'.format(z[4]).replace(",", " "),
                               line_dash = 'dot', line_color = '#ffa306')
                 ]


my_title = "Приземные концентрации на разных высотах"
mv_title_x = "$$k, $$"
mv_title_y = "$$z, м$$"
file_name = "diary_tau"
xaxis_min = 0.
xaxis_max = 0.1
mv_legend_x = 1.
mv_legend_anchor = "right"

mv_graphics_stat(graphs_visible, my_title,
                 mv_title_x, mv_title_y,
                 file_name,
                 xaxis_min, xaxis_max,
                 mv_legend_x, mv_legend_anchor)